First, let's create a synthetic dataset for classification:

In [1]:
from collections import Counter
from sklearn.datasets import make_classification
# create dataframe
X, y = make_classification(n_samples=1000,
                           n_features=20,
                           n_informative=15,
                           n_redundant=5,
                           random_state=1)
# print the data classes info
print(f'''Main dataframe:
Number of samples: {X.shape[0]}
Number of features: {X.shape[1]}
Samples by class:''')
counter = Counter(y)
for k, v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))

# Main dataframe:
# Number of samples: 1000
# Number of features: 20
# Samples by class:
# Class=0, Count=501, Percentage=50.1%
# Class=1, Count=499, Percentage=49.9%

Then we split the data into training and test parts:

In [2]:
from sklearn.model_selection import train_test_split
# split data to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, stratify=y, random_state=1)
# print the data classes info
print(f'''\nTrain dataframe:
Number of samples: {X_train.shape[0]}
Number of features: {X_train.shape[1]}
Samples by class:''')
counter = Counter(y_train)
for k, v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%d, Count=%d, Percentage=%.1f%%' % (k, v, per))

# Train dataframe:
# Number of samples: 500
# Number of features: 20
# Samples by class:
# Class=0, Count=251, Percentage=50.2%
# Class=1, Count=249, Percentage=49.8%

And now let's write a function for training and evaluating classifiers:

In [3]:
import time
from sklearn.metrics import accuracy_score, confusion_matrix
# define predictor function
def predictor(model):
    tic = time.perf_counter()
    # fit classifier
    model.fit(X_train, y_train)
    # get prediction
    y_pred = model.predict(X_test)
    # check results
    accuracy = accuracy_score(y_test, y_pred)
    matrix = confusion_matrix(y_test, y_pred)
    # show results
    print(f'''\n{type(model).__name__} results:
Accuracy:{accuracy * 100: 0.1f}%
{matrix}''')
    toc = time.perf_counter()
    print(f"Processed in {toc - tic: 0.4f} seconds")

It's time to write some code that demonstrates how well Decision Trees perform with classification:

In [4]:
from sklearn.tree import DecisionTreeClassifier
# call Decision Tree
predictor(DecisionTreeClassifier(random_state=1))

# DecisionTreeClassifier results:
# Accuracy: 78.0%
# [[200  50]
#  [ 60 190]]
# Processed in  0.0157 seconds

Bagging Ensemble.
Now let's see how much this approach is better than a single tree:

In [5]:
from sklearn.ensemble import BaggingClassifier
# call Bagging Trees
predictor(BaggingClassifier(random_state=1))

# BaggingClassifier results:
# Accuracy: 87.8%
# [[225  25]
#  [ 36 214]]
# Processed in  0.1033 seconds

Can Random Forest give even better results?

In [6]:
from sklearn.ensemble import RandomForestClassifier
# call Random Forest
predictor(RandomForestClassifier(random_state=1))

# RandomForestClassifier results:
# Accuracy: 91.2%
# [[234  16]
#  [ 28 222]]
# Processed in  0.2108 seconds

Extremely Randomized Trees.
Even better?

In [7]:
from sklearn.ensemble import ExtraTreesClassifier
# call Extra Trees
predictor(ExtraTreesClassifier(random_state=1))

# ExtraTreesClassifier results:
# Accuracy: 93.0%
# [[238  12]
#  [ 23 227]]
# Processed in  0.1247 seconds

Adaptive Boosting (AdaBoost).
Sounds cool, but does it really work?

In [8]:
from sklearn.ensemble import AdaBoostClassifier
# call Adaptive Boosting
predictor(AdaBoostClassifier(random_state=1))

# AdaBoostClassifier results:
# Accuracy: 85.2%
# [[220  30]
#  [ 44 206]]
# Processed in  0.1048 seconds

Gradient Boosting.
Let's see how much better this approach is:

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
# call Gradient Boosting
predictor(GradientBoostingClassifier(random_state=1))

# GradientBoostingClassifier results:
# Accuracy: 91.4%
# [[230  20]
#  [ 23 227]]
# Processed in  0.2346 seconds

Extreme Gradient Boosting (XGBoost).
Let's check:

In [10]:
from xgboost import XGBClassifier
# call Extreme Gradient Boosting
predictor(XGBClassifier(random_state=1))

# XGBClassifier results:
# Accuracy: 92.2%
# [[235  15]
#  [ 24 226]]
# Processed in  1.4683 seconds

Histogram-Based Gradient Boosting.
Let's see how it copes with our small dataset:

In [11]:
from sklearn.ensemble import HistGradientBoostingClassifier
# call Histogram-Based Gradient Boosting
predictor(HistGradientBoostingClassifier(random_state=1))

# HistGradientBoostingClassifier results:
# Accuracy: 92.4%
# [[235  15]
#  [ 23 227]]
# Processed in  0.3195 seconds

We will select all models that have shown more than 90% accuracy. There are 5 of them:

In [12]:
estimators = [('rf', RandomForestClassifier(random_state=1)),
              ('et', ExtraTreesClassifier(random_state=1)),
              ('gb', GradientBoostingClassifier(random_state=1)),
              ('xgb', XGBClassifier(random_state=1)),
              ('hgb', HistGradientBoostingClassifier(learning_rate=1))]

Stack of estimators with a final classifier (Stacking).
And now let's find out if the ensemble of ensembles can give the result above:

In [13]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
# call Stacked generalization
predictor(StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()))

# StackingClassifier results:
# Accuracy: 92.6%
# [[239  11]
#  [ 26 224]]
# Processed in  2.8881 seconds

Hard Voting Classifier.
Can this approach be better than Stacking? Let's find out!

In [14]:
from sklearn.ensemble import VotingClassifier
# call Soft Majority Rule classifier
predictor(VotingClassifier(estimators=estimators, voting='hard'))

# VotingClassifier results:
# Accuracy: 92.6%
# [[238  12]
#  [ 25 225]]
# Processed in  0.5794 seconds

Weighted Average Probabilities (Soft Voting Classifier).
It seems that such a classification would be more accurate:

In [15]:
from sklearn.ensemble import VotingClassifier
# call Soft Majority Rule classifier
predictor(VotingClassifier(estimators=estimators, voting='soft'))

# VotingClassifier results:
# Accuracy: 93.2%
# [[239  11]
#  [ 23 227]]
# Processed in  0.5797 seconds